## SpatioTemporal Asset Catalog (STAC) Creation Script
This code will create a STAC catalog by crawling over folders in the Cyverse Data Store and looking for geospatial assets (e.g., orthomosaics, point clouds, DEMs). It is specifically built to create a STAC catalog for the Open Forest Observatory (OFO) project. 

Please launch 'JupyterLab_Geospatial' VICE app in Cyverse Discovery Environment
Open this Jupyter Notebook `STAC_creation_OFO.ipynb`
Change the Kernel to `osgeo`
Use at least 16gb of RAM

In [8]:
# Install all dependencies with specific versions
%pip install \
  geopandas \
  shapely==1.8.5 \
  rasterio==1.3.4 \
  laspy==2.2.0 \
  pystac==1.6.0 \
  pyproj==3.4.0 \
  laspy[lazrs]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from pathlib import Path
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, mapping
import numpy as np

# Raster / LAZ imports
import rasterio
from rasterio.warp import transform_bounds
from rasterio.coords import BoundingBox
from osgeo import ogr, osr
import laspy

# Optional: pystac for building STAC objects more systematically
import pystac
from pystac import (CatalogType, MediaType)
from pystac.extensions.eo import EOExtension
from pystac.extensions.projection import ProjectionExtension
from datetime import datetime

In [2]:
def trun_n_d(num, n):
    """
    Truncate spatial resolution to 'n' decimal places.
    """
    num_s = str(num)
    if 'e' in num_s or 'E' in num_s:
        return f'{num:.{n}f}'
    i, p, d = num_s.partition('.')
    return '.'.join([i, (d + '0'*n)[:n]])

In [3]:
def spatial_resolution(raster):
    """
    Extracts the XY Pixel Size from a rasterio Dataset.
    Only works if imagery has a map projection, otherwise returns (0.00, 0.00).
    """
    t = raster.transform
    x = t[0]
    y = -t[4]
    x_trunc = trun_n_d(x, 3)
    y_trunc = trun_n_d(y, 3)
    return x_trunc, y_trunc

In [4]:
def get_bbox_and_footprint(dataset):
    """
    Returns:
      bbox: [left, bottom, right, top] in EPSG:4326
      footprint: shapely polygon geometry (as a geo-interface dict) in EPSG:4326
    """
    # Original bounding box in native CRS
    bounds = dataset.bounds
    
    # Transform bounds to EPSG:4326
    bounds_4326 = transform_bounds(dataset.crs, 'EPSG:4326',
                                   bounds.left, bounds.bottom, 
                                   bounds.right, bounds.top)
    # Convert to a named bounding box
    bounds_4326 = BoundingBox(*bounds_4326)
    
    # Build a simple list
    bbox = [bounds_4326.left, bounds_4326.bottom, bounds_4326.right, bounds_4326.top]
    
    # Create a polygon footprint
    footprint = Polygon([
        (bbox[0], bbox[1]),  # left, bottom
        (bbox[0], bbox[3]),  # left, top
        (bbox[2], bbox[3]),  # right, top
        (bbox[2], bbox[1])   # right, bottom
    ])
    return bbox, mapping(footprint)

In [5]:
def tif_get_spatial_info(tif_file_path: Path):
    """
    Extract bounding box, footprint, resolution, CRS, shape, etc. from a GeoTIFF
    in a single function.
    Returns a dict of relevant properties or None if error.
    """
    try:
        with rasterio.open(tif_file_path) as ds:
            bbox, footprint = get_bbox_and_footprint(ds)
            x_res, y_res = spatial_resolution(ds)
            
            # Raster shape
            height, width = ds.shape
            # Attempt to retrieve EPSG integer if possible
            epsg_code = ds.crs.to_epsg() if ds.crs else None
            
            return {
                "path": str(tif_file_path),
                "bbox": bbox,
                "footprint": footprint,
                "x_res": x_res,
                "y_res": y_res,
                "width": width,
                "height": height,
                "epsg": epsg_code,
                "media_type": MediaType.COG  # Or 'image/tiff'
            }
    except Exception as e:
        print(f"Error reading {tif_file_path} - skipping. Error: {e}")
        return None

def get_las_bbox_footprint(las_srs_wkt: str, boundary_wkt: str, target_epsg: int = 4326):
    """
    Returns bounding box, footprint geometry, and EPSG for a LAZ file.
    
    boundary_wkt: WKT polygon of the LAZ boundary in native CRS.
    las_srs_wkt: The coordinate system WKT.
    
    Returns:
      return_bbox: (minX, minY, maxX, maxY) in EPSG:4326
      footprint: shapely Polygon geometry in EPSG:4326
      geom_epsg: original EPSG code (if found)
    """
    bounds_polygon = ogr.CreateGeometryFromWkt(boundary_wkt)

    # Parse source SRS
    src_srs = osr.SpatialReference()
    src_srs.ImportFromWkt(las_srs_wkt)

    geom_epsg = None
    try:
        geom_epsg = int(src_srs.GetAttrValue('AUTHORITY', 1))
    except:
        pass  # Not all LAS have an authority code

    # If needed, transform to target EPSG
    if geom_epsg != target_epsg:
        dst_srs = osr.SpatialReference()
        dst_srs.ImportFromEPSG(target_epsg)

        transform = osr.CoordinateTransformation(src_srs, dst_srs)
        transl_polygon = bounds_polygon.Clone()
        transl_polygon.Transform(transform)
        bounds_polygon = transl_polygon

    # Envelope -> (minX, maxX, minY, maxY)
    envelope = bounds_polygon.GetEnvelope()
    # Re-arrange to (minX, minY, maxX, maxY)
    return_bbox = (envelope[0], envelope[2], envelope[1], envelope[3])

    # Construct shapely polygon from that envelope
    footprint = Polygon([
        (return_bbox[0], return_bbox[1]),  # left, bottom
        (return_bbox[0], return_bbox[3]),  # left, top
        (return_bbox[2], return_bbox[3]),  # right, top
        (return_bbox[2], return_bbox[1])   # right, bottom
    ])

    return return_bbox, mapping(footprint), geom_epsg

def las_get_spatial_info(laz_file_path: Path):
    """
    Extract bounding box, footprint, approximate resolution, etc. from a LAZ file.
    Returns a dict or None on error.
    """
    try:
        # Open LAZ using laspy
        with laspy.open(laz_file_path) as f:
            header = f.header
            # bounding box in native CRS from LAS header
            minx, maxx = header.mins[0], header.maxs[0]
            miny, maxy = header.mins[1], header.maxs[1]
            # You can also check the 'z' dimension if needed

            # We'll attempt to get the WKT from the header
            las_srs_wkt = header.vlr().coordinate_system_description
            # Fallback if no WKT found
            if not las_srs_wkt:
                # If no SRS, skip or assume EPSG:4326
                las_srs_wkt = osr.SpatialReference()
                las_srs_wkt.ImportFromEPSG(4326)
                las_srs_wkt = las_srs_wkt.ExportToWkt()

            # Build WKT polygon from bounding box
            wkt_poly = f"POLYGON (({minx} {miny}, {minx} {maxy}, {maxx} {maxy}, {maxx} {miny}, {minx} {miny}))"
            bbox_4326, footprint_4326, geom_epsg = get_las_bbox_footprint(
                las_srs_wkt=las_srs_wkt,
                boundary_wkt=wkt_poly,
                target_epsg=4326
            )
            
            # Approx resolution can be guessed or omitted. 
            # E.g., many point clouds don't have a single GSD. We'll omit advanced logic,
            # or just store "N/A".
            # If you really want a rough GSD, you'd have to define your approach. 
            # We'll do "N/A" for now or store them as strings.
            x_res, y_res = "N/A", "N/A"

            return {
                "path": str(laz_file_path),
                "bbox": list(bbox_4326),  # Convert to list
                "footprint": footprint_4326,
                "x_res": x_res,
                "y_res": y_res,
                "epsg": geom_epsg if geom_epsg else 4326,
                "media_type": "application/octet-stream"  # Or "application/las" if you prefer
            }

    except Exception as e:
        print(f"Error reading {laz_file_path} - skipping. Error: {e}")
        return None

In [ ]:
# 3.1 Define your paths
#gpkg_path = Path("/data-store/iplant/home/shared/ofo/public/metadata/all-mission-polygons-w-metadata.gpkg")# full dataset
gpkg_path = Path("/data-store/iplant/home/jgillan/ofo_mini/metadata/all-mission-polygons-w-metadata.gpkg") #mini dataset for fast testing
#missions_root = Path("/data-store/iplant/home/shared/ofo/public/missions") #full dataset
missions_root = Path("/data-store/iplant/home/jgillan/ofo_mini/missions") #mini dataset

# 3.2 Read the GPKG into a GeoDataFrame
#     We assume geometry is already EPSG:4326
gdf = gpd.read_file(gpkg_path)
print(f"Read {len(gdf)} missions from GPKG.")

In [ ]:
# 3.3 We only keep missions where we have an ID and a datetime
gdf = gdf.dropna(subset=["dataset_id", "earliest_datetime_local_derived"])
gdf["id"] = gdf["dataset_id"].astype(str)

# Convert that local derived datetime string to a proper ISO 8601
gdf["datetime"] = pd.to_datetime(gdf["earliest_datetime_local_derived"])

# 3.4 Create empty lists to store STAC items and a place to track overall min/max for bounding box
all_items = []

# For temporal extent across the entire collection
all_datetimes = []

# We'll store numeric extremes for the global bounding box
min_x_list, min_y_list, max_x_list, max_y_list = [], [], [], []

In [ ]:
# 3.5 Known asset filenames or patterns
#    We'll search for these in each mission's "processed/full" folder
TIF_FILENAMES = [
    "orthomosaic.tif", 
    "chm-mesh.tif", 
    "chm-ptcloud.tif", 
    "dsm-mesh.tif", 
    "dsm-ptcloud.tif", 
    "dtm-ptcloud.tif"
]
LAZ_FILENAMES = [
    "points.laz"
]

In [ ]:
# 3.6 Iterate over each row in the GPKG
CYVERSE_BASE = "https://data.cyverse.org/dav-anon"

for idx, row in gdf.iterrows():
    mission_id = row["id"]  # e.g. "000421"
    mission_datetime = row["datetime"].tz_localize(None)  # remove time zone if any
    
    # Convert mission_datetime to ISO 8601 string and append 'Z'
    mission_datetime_str = mission_datetime.isoformat() + "Z"
    
    mission_folder = missions_root / mission_id

    # 3.6.1 Check if there's a subfolder that starts with "processed"
    #       Because the structure might be mission_id -> processed* -> full
    #       We'll do a quick search:
    if not mission_folder.exists():
        print(f"Mission folder does not exist: {mission_folder}. Skipping...")
        continue
    else:
        print(f"Mission folder found: {mission_folder}")
    
    
    # Attempt to find path: mission_id/processed*/full
    processed_path = None
    for item in mission_folder.iterdir():
        if item.is_dir() and item.name.startswith("processed"):
            # We found something like processed or processed_whatever
            processed_path = item
            break
    
    if not processed_path:
        # No processed folder found, skip
        continue
    
    full_path = processed_path / "full"
    if not full_path.exists():
        # No full folder found
        continue
    
    
    # Pull the 'Aircraft' column from the row (assuming it always exists)
    aircraft_type = row["aircraft_model_name"]  # e.g. "DJI Phantom 4 Pro"
    
    
    # 3.6.2 Look for the known TIF or LAZ assets
    # We'll store data about each found asset in a dictionary
    assets_info = {}
    
    # Collect bounding geometries to union them
    union_poly = None
    union_bbox = [9999, 9999, -9999, -9999]  # [minx, miny, maxx, maxy]
    
    # TIFs
    for tif_name in TIF_FILENAMES:
        candidate_path = full_path / tif_name
        if candidate_path.exists():
            # Extract info
            tif_info = tif_get_spatial_info(candidate_path)
            if tif_info:
                # Update union bounding geometry
                minx, miny, maxx, maxy = tif_info["bbox"]
                
                # Expand union_bbox
                if minx < union_bbox[0]: union_bbox[0] = minx
                if miny < union_bbox[1]: union_bbox[1] = miny
                if maxx > union_bbox[2]: union_bbox[2] = maxx
                if maxy > union_bbox[3]: union_bbox[3] = maxy
                
                #Create url for assets
                posix_str = candidate_path.as_posix()
                cyverse_relative_path = posix_str.replace("/data-store", "")
                asset_href = CYVERSE_BASE + cyverse_relative_path
                
                # We'll store the asset by a key that matches the file name, or something descriptive
                asset_key = tif_name.replace(".tif", "")
                assets_info[asset_key] = {
                    "href": asset_href,
                    "type": tif_info["media_type"],
                    "roles": ["data"],  # Or something else 
                    "proj:epsg": tif_info["epsg"],
                    "gsd": [float(tif_info["x_res"]), float(tif_info["y_res"])],
                }
    
    # LAZ
    for laz_name in LAZ_FILENAMES:
        candidate_path = full_path / laz_name
        if candidate_path.exists():
            #Create url for assets
                posix_str = candidate_path.as_posix()
                cyverse_relative_path = posix_str.replace("/data-store", "")
                asset_href = CYVERSE_BASE + cyverse_relative_path
                
                asset_key = laz_name.replace(".laz", "")
                assets_info[asset_key] = {
                    "href": asset_href,
                    "type": "application/vnd.laszip+copc",
                    "roles": ["data"],
                    "proj:epsg": tif_info["epsg"],    
                }
    
    # If we found no assets, skip (per your requirement)
    if not assets_info:
        continue

    # 3.6.3 Build union geometry from union_bbox
    if union_bbox[0] == 9999 or union_bbox[1] == 9999:
        # Means we never updated it
        # Skip this mission if no bounding geometry
        continue
    
    # Create an item-level bounding box
    item_bbox = union_bbox  # [minX, minY, maxX, maxY]

    # Create a polygon footprint
    item_footprint = Polygon([
        (item_bbox[0], item_bbox[1]),  # left, bottom
        (item_bbox[0], item_bbox[3]),  # left, top
        (item_bbox[2], item_bbox[3]),  # right, top
        (item_bbox[2], item_bbox[1])   # right, bottom
    ])
    
    # 3.6.4 Update global bounding box and time
    min_x_list.append(item_bbox[0])
    min_y_list.append(item_bbox[1])
    max_x_list.append(item_bbox[2])
    max_y_list.append(item_bbox[3])
    all_datetimes.append(mission_datetime)

    # 3.6.5 Build the STAC Item dictionary 
    #       If you prefer raw dict approach (rather than PySTAC):
    item_dict = {
        "type": "Feature",
        "collection": "Open Forest Observatory",
        "stac_extensions": [
                "https://stac-extensions.github.io/projection/v1.0.0/schema.json",
                "https://stac-extensions.github.io/scientific/v1.0.0/schema.json"
        ],
        "stac_version": "1.0.0",
        "id": mission_id,
        "properties": {
            "datetime": mission_datetime_str,
            "license": "CC-BY-SA-4.0",
            "platform": aircraft_type
        },
        "geometry": mapping(item_footprint),  # geojson geometry
        "bbox": item_bbox,
        "links": [
            {
            "rel": "preview",
            "href": "https://raw.githubusercontent.com/cyverse-gis/cyverse-stac/refs/heads/main/data/forest.png",
            "type": "image/png",
            "title": "Item Thumbnail"
        }
        ],  
        "assets": assets_info
    }
    
    all_items.append(item_dict)

In [11]:
# -----------------------------
# 4. Build the STAC Collection
# -----------------------------
if len(all_items) == 0:
    print("No STAC Items found! Exiting with no outputs.")
    # You can decide if you want to write empty files or not
    raise SystemExit

collection_bbox = [
    float(np.min(min_x_list)), 
    float(np.min(min_y_list)), 
    float(np.max(max_x_list)), 
    float(np.max(max_y_list))
]

# For temporal extent
min_date = min(all_datetimes)
max_date = max(all_datetimes)

# Append 'Z' to each datetime to indicate UTC (Zulu time)
min_date_str = min_date.isoformat() + "Z"
max_date_str = max_date.isoformat() + "Z"

collection_dict = {
    "type": "Collection",
    "stac_version": "1.0.0",
    "id": "Open Forest Observatory",
    "description": "Drone Based Forest Mapping for Forest Ecology & Management",
    "license": "CC-BY-SA-4.0",
    "links": [
        {
      "rel": "preview",
      "href": "https://raw.githubusercontent.com/cyverse-gis/cyverse-stac/refs/heads/main/data/forest.png",
      "type": "image/png",
      "title": "Collection Thumbnail"
    }
    ],
    "extent": {
        "spatial": {
            "bbox": [collection_bbox]
        },
        "temporal": {
            "interval": [[
                min_date_str,
                max_date_str
            ]]
        }
    }
}

In [12]:
# -----------------------------
# 5. Export to JSON
# -----------------------------

# 5.1 index.geojson: FeatureCollection of Items
index_geojson = {
    "type": "FeatureCollection",
    "features": all_items
}

with open("/data-store/iplant/home/jgillan/index12.geojson", "w", encoding="utf-8") as f:
    json.dump(index_geojson, f, indent=2)

# 5.2 collection.json
with open("/data-store/iplant/home/jgillan/collection12.json", "w", encoding="utf-8") as f:
    json.dump(collection_dict, f, indent=2)

print("STAC Collection (collection.json) and Items (index.geojson) have been created!")


STAC Collection (collection.json) and Items (index.geojson) have been created!
